<a href="https://colab.research.google.com/github/kimsojeong1225/DeepLearning/blob/master/Renet50%2B%ED%8A%B9%EC%84%B1%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import tensorflow.keras

In [0]:
from tensorflow.keras.applications import resnet50

In [0]:
cd drive

/content/drive


In [0]:
cd My\ Drive

/content/drive/My Drive


In [0]:
cd cats_and_dogs_small/

/content/drive/My Drive/cats_and_dogs_small


In [0]:
train_cats_dir='./train/cats'
train_dogs_dir='./train/dogs'
validation_cats_dir='./validation/cats'
validation_dogs_dir='./validation/dogs'
test_cats_dir='./test/cats'
test_dogs_dir='./test/dogs'
train_dir='./train'
validation_dir='./validation'
test_dir='./test'
base_dir='./'

In [0]:
conv_base=tensorflow.keras.applications.ResNet50(include_top=False,
                                        weights='imagenet',
                                        input_shape=(150,150,3))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
94773248/94765736 [==============================] - 2s 0us/step


In [0]:
conv_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 75, 75, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [0]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [0]:
model=models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 5, 5, 2048)        23587712  
_________________________________________________________________
flatten_2 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               13107456  
_________________________________________________________________
dense_6 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 36,728,193
Trainable params: 36,675,073
Non-trainable params: 53,120
_________________________________________________________________


In [0]:
print('conv_base를 동결하기 전 훈련되는 가중치의 수: ',len(model.trainable_weights))
conv_base.trainable=False
print('conv_base를 동결한 후 훈련되는 가중치의 수: ',len(model.trainable_weights))

conv_base를 동결하기 전 훈련되는 가중치의 수:  218
conv_base를 동결한 후 훈련되는 가중치의 수:  6


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [0]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary'
)
validation_generator=test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary' 
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [0]:
conv_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 75, 75, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [0]:
#conv_base 동결 해제
conv_base.trainable = True

set_trainable=False                                   
for layer in conv_base.layers:
  if layer.name == 'conv5_block1_1_conv':
    set_trainable = True
  if set_trainable:
    print(layer.name,': True')
    layer.trainable=True
  else:
    layer.trainable=False
    print(layer.name,': False')

input_1 : False
conv1_pad : False
conv1_conv : False
conv1_bn : False
conv1_relu : False
pool1_pad : False
pool1_pool : False
conv2_block1_1_conv : False
conv2_block1_1_bn : False
conv2_block1_1_relu : False
conv2_block1_2_conv : False
conv2_block1_2_bn : False
conv2_block1_2_relu : False
conv2_block1_0_conv : False
conv2_block1_3_conv : False
conv2_block1_0_bn : False
conv2_block1_3_bn : False
conv2_block1_add : False
conv2_block1_out : False
conv2_block2_1_conv : False
conv2_block2_1_bn : False
conv2_block2_1_relu : False
conv2_block2_2_conv : False
conv2_block2_2_bn : False
conv2_block2_2_relu : False
conv2_block2_3_conv : False
conv2_block2_3_bn : False
conv2_block2_add : False
conv2_block2_out : False
conv2_block3_1_conv : False
conv2_block3_1_bn : False
conv2_block3_1_relu : False
conv2_block3_2_conv : False
conv2_block3_2_bn : False
conv2_block3_2_relu : False
conv2_block3_3_conv : False
conv2_block3_3_bn : False
conv2_block3_add : False
conv2_block3_out : False
conv3_block1_1_c

In [0]:
print('conv_base일부를 동결해제한 후 가중치 수: ',len(model.trainable_weights))

conv_base일부를 동결해제한 후 가중치 수:  46


In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 5, 5, 2048)        23587712  
_________________________________________________________________
flatten_2 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               13107456  
_________________________________________________________________
dense_6 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 36,728,193
Trainable params: 28,116,481
Non-trainable params: 8,611,712
_________________________________________________________________


In [0]:
history=model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=3,
    validation_data=validation_generator,
    validation_steps=50
)

Epoch 1/3
100/100 [==============================] - 463s 5s/step - loss: 0.1256 - acc: 0.9515 - val_loss: 0.7317 - val_acc: 0.5000
Epoch 2/3
100/100 [==============================] - 460s 5s/step - loss: 0.1029 - acc: 0.9615 - val_loss: 0.7383 - val_acc: 0.5000
Epoch 3/3
100/100 [==============================] - 460s 5s/step - loss: 0.1067 - acc: 0.9565 - val_loss: 0.7222 - val_acc: 0.5000


In [0]:
conv_base2=tensorflow.keras.applications.ResNet50(include_top=False,
                                        weights=None,
                                        input_shape=(150,150,3))

In [0]:
model2=models.Sequential()
model2.add(conv_base2)
model2.add(layers.Flatten())
model2.add(layers.Dense(256,activation='relu'))
model2.add(layers.Dense(128,activation='relu'))
model2.add(layers.Dense(1,activation='sigmoid'))
model2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 5, 5, 2048)        23587712  
_________________________________________________________________
flatten_3 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               13107456  
_________________________________________________________________
dense_9 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 129       
Total params: 36,728,193
Trainable params: 36,675,073
Non-trainable params: 53,120
_________________________________________________________________


In [0]:
print('conv_base를 동결하기 전 훈련되는 가중치의 수: ',len(model2.trainable_weights))
conv_base2.trainable=False
print('conv_base를 동결한 후 훈련되는 가중치의 수: ',len(model2.trainable_weights))

conv_base를 동결하기 전 훈련되는 가중치의 수:  218
conv_base를 동결한 후 훈련되는 가중치의 수:  6


In [0]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=0.1,
      horizontal_flip=True,
      fill_mode='nearest')

In [0]:
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [0]:
model2.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])


In [0]:
history = model2.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=3,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

Epoch 1/3
Epoch 1/3
100/100 - 348s - loss: 0.7797 - acc: 0.5245 - val_loss: 0.6933 - val_acc: 0.5010
Epoch 2/3
Epoch 1/3
100/100 - 346s - loss: 0.7292 - acc: 0.5335 - val_loss: 0.6933 - val_acc: 0.4840
Epoch 3/3
Epoch 1/3
100/100 - 345s - loss: 0.7184 - acc: 0.5310 - val_loss: 0.6931 - val_acc: 0.5030


In [0]:
conv_base3=tensorflow.keras.applications.ResNet50(include_top=False,
                                        weights='imagenet',
                                        input_shape=(150,150,3))

In [0]:
model3=models.Sequential()
model3.add(conv_base2)
model3.add(layers.Flatten())
model3.add(layers.Dense(256,activation='relu'))
model3.add(layers.Dense(128,activation='relu'))
model3.add(layers.Dense(1,activation='sigmoid'))
model3.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 5, 5, 2048)        23587712  
_________________________________________________________________
flatten_4 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               13107456  
_________________________________________________________________
dense_12 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 129       
Total params: 36,728,193
Trainable params: 13,140,481
Non-trainable params: 23,587,712
_________________________________________________________________


In [0]:
conv_base3.trainable=False

In [0]:
model3.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])


In [0]:
history = model3.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=3,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

Epoch 1/3
Epoch 1/3
100/100 - 352s - loss: 0.8645 - acc: 0.5000 - val_loss: 0.6932 - val_acc: 0.5050
Epoch 2/3
Epoch 1/3
100/100 - 346s - loss: 0.7468 - acc: 0.5235 - val_loss: 0.6932 - val_acc: 0.4850
Epoch 3/3
Epoch 1/3
100/100 - 345s - loss: 0.7208 - acc: 0.5260 - val_loss: 0.6930 - val_acc: 0.5030


# 데이터 증식 x


In [0]:
conv_base=tensorflow.keras.applications.ResNet50(include_top=False,
                                        weights='imagenet',
                                        input_shape=(240,240,3))

In [0]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)
batch_size=20

train_cats_dir='./train/cats'
train_dogs_dir='./train/dogs'
validation_cats_dir='./validation/cats'
validation_dogs_dir='./validation/dogs'
test_cats_dir='./test/cats'
test_dogs_dir='./test/dogs'
train_dir='./train'
validation_dir='./validation'
test_dir='./test'
base_dir='./'

def extract_features(directory,sample_count):
  features = np.zeros(shape=(sample_count,8,8,2048))
  labels=np.zeros(shape=(sample_count))
  generator = datagen.flow_from_directory(
      directory,
      target_size=(240,240),
      batch_size=batch_size,
      class_mode='binary'
  )
  
  
  
  i=0
  for inputs_batch,labels_batch in generator:
    features_batch = conv_base.predict(inputs_batch)
    features[i*batch_size : (i+1)*batch_size]=features_batch
    i+=1
    if i * batch_size >= sample_count:
      break     #제너레이터는 루프 안에서 무한하게 데이터를 만들어내므로 모든 이미지를 한 번씩 처리하고 나면 중지
  return features,labels

train_features,train_labels=extract_features(train_dir,2000)
validation_features,validation_labels=extract_features(validation_dir,1000)
test_features,test_labels=extract_features(test_dir,1000)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [0]:
model3.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

In [0]:
history = model3.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=3,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

Epoch 1/3
Epoch 1/3
100/100 - 352s - loss: 0.7025 - acc: 0.5505 - val_loss: 0.6932 - val_acc: 0.4860
Epoch 2/3
Epoch 1/3
100/100 - 345s - loss: 0.6890 - acc: 0.5730 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 3/3
Epoch 1/3
100/100 - 343s - loss: 0.6822 - acc: 0.5700 - val_loss: 0.6931 - val_acc: 0.5090


In [0]:
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(1,len(acc)+1)

plt.plot(epochs,acc,'bo',label='Training acc')
plt.plot(epochs,val_acc,'b',label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs,loss,'bo',label='Training loss')
plt.plot(epochs,val_loss,'b',label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: ignored

In [0]:
def smooth_curve(points,factor=0.8):
  for point in points
  if smoothed_points:
    previous=smoothed_points[-1]
    smoothed_points.append(previous * factor + point * (1-factor))
  else:
    smoothed_points.append(point)
  return smoothed_points

plt.plot(epochs,
         smooth_curve(acc),'bo',label='Smoothed training acc')
plt.plot(epochs,
         smooth_curve(val_acc),'b',label='Smoothed validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs,
         smooth_curve(loss),'bo',label='Smoothed training loss')
plt.plot(epochs,
         smooth_curve(val_loss),'b',label='Smoothed validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()


In [0]:
test_generator = test.datagen.flow_from_directory(
    test_dir,
    target_size(150,150),
    batch_size=20,
    class_mode='binary'
)
test_loss,test_acc=model.evaluate_generator(test_generator,steps=50)
print('test acc:',test_acc)